In [2]:
import pandas as pd

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier

In [4]:
from sklearn.model_selection import cross_val_score

In [7]:
soloq_df = pd.read_excel('8-18_dataset.xlsx')

In [12]:
slo_df = pd.read_excel('slo_dataset.xlsx')

In [13]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [14]:
newdf = slo_df.select_dtypes(include=numerics)
newdf2 = soloq_df.select_dtypes(include=numerics)

In [140]:
newdf.drop([col for col in newdf.columns if 'Id' in col], axis=1, inplace=True)
newdf2.drop([col for col in newdf2.columns if 'Id' in col], axis=1, inplace=True)

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [141]:
perkp_df = slo_df.perkPrimaryStyle_name.str.get_dummies()
perks_df = slo_df.perkSubStyle_name.str.get_dummies()

perkp_df2 = soloq_df.perkPrimaryStyle_name.str.get_dummies()
perks_df2 = soloq_df.perkSubStyle_name.str.get_dummies()

In [142]:
role_dum = slo_df.role.str.get_dummies()
lane_dum = slo_df.lane.str.get_dummies()

role_dum2 = soloq_df.role.str.get_dummies()
lane_dum2 = soloq_df.lane.str.get_dummies()

In [143]:
lane_dum2.drop('NONE', axis=1, inplace=True)

In [144]:
champ_df = slo_df.champ_name.str.get_dummies()

champ_df2 = soloq_df.champ_name.str.get_dummies()

In [145]:
all_num_df = pd.concat([newdf, perkp_df, perks_df, champ_df, role_dum, lane_dum], axis=1)
all_num_df2 = pd.concat([newdf2, perkp_df2, perks_df2, champ_df2, role_dum2, lane_dum2], axis=1)

# all_num_df = pd.concat([perkp_df, perks_df, champ_df, role_dum, lane_dum], axis=1)
# all_num_df2 = pd.concat([perkp_df2, perks_df2, champ_df2, role_dum2, lane_dum2], axis=1)

# all_num_df = pd.concat([newdf, perkp_df, perks_df, champ_df, role_dum, lane_dum], axis=1)
# all_num_df2 = pd.concat([newdf2, perkp_df2, perks_df2, champ_df2, role_dum2, lane_dum2], axis=1)

## Fill NA values 

In [146]:
fill_0_cols = all_num_df.T.ix[all_num_df.isnull().any().tolist()].T.columns.tolist()

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [147]:
all_num_df[fill_0_cols] = all_num_df[fill_0_cols].fillna(0).astype(int)

In [148]:
all_num_df2[fill_0_cols] = all_num_df2[fill_0_cols].fillna(0).astype(int)

In [149]:
fill_mean_cols = [col for col in all_num_df2.columns if 'tt' in col]

In [150]:
all_num_df2[fill_mean_cols] = all_num_df2[fill_mean_cols].fillna(all_num_df2[fill_mean_cols].mean())

In [151]:
fill_0_cols_2 = all_num_df2.T.ix[all_num_df2.isnull().any().tolist()].T.columns.tolist()

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [152]:
all_num_df2[fill_0_cols_2] = all_num_df2[fill_0_cols_2].fillna(0)

## Fill missing cols 

In [153]:
missing_cols = set(all_num_df2.columns) - set(all_num_df.columns)

In [154]:
all_num_df = all_num_df.assign(**{col: 0 for col in missing_cols})

##  Predict

In [155]:
trainers = all_num_df
predictors = all_num_df2

In [129]:
labels = slo_df.position

In [130]:
model1 = DecisionTreeClassifier()
model2 = BernoulliNB()
model3 = MLPClassifier()
model4 = SVC()
model5 = BaggingClassifier()
model6 = GradientBoostingClassifier() # 99%
model7 = RandomForestClassifier() # 

In [96]:
models = [model1, model2, model3, model4, model5, model6, model7]

In [97]:
models_results = pd.concat([pd.DataFrame(cross_val_score(estimator=model, cv=5, X=trainers, y=labels)).T for model in models])

In [98]:
models_results['model'] = ['DecisionTreeClassifier', 'BernoulliNB', 'MLPClassifier', 'SVC', 'BaggingClassifier', 'GradientBoostingClassifier', 'RandomForestClassifier']

In [99]:
models_results['mean'] = models_results[[0, 1, 2, 3, 4]].T.mean()

In [100]:
models_results['std'] = models_results[[0, 1, 2, 3, 4]].T.std()

In [101]:
models_results.reset_index(drop=True)

,0,1,2,3,4,model,mean,std
0,0.9425,0.9125,0.9250,0.9025,0.9150,DecisionTreeClassifier,0.9195,0.015145
1,0.9200,0.8950,0.9000,0.8825,0.8800,BernoulliNB,0.8955,0.016047
2,0.9500,0.9525,0.9650,0.9500,0.9575,MLPClassifier,0.9550,0.006374
3,0.8550,0.8100,0.7975,0.7800,0.7975,SVC,0.8080,0.028362
4,0.9250,0.9250,0.9200,0.8975,0.9150,BaggingClassifier,0.9165,0.011402
5,0.9375,0.9450,0.9450,0.9375,0.9450,GradientBoostingClassifier,0.9420,0.004108
6,0.9300,0.9150,0.9150,0.8950,0.9175,RandomForestClassifier,0.9145,0.012550


In [156]:
model3.fit(X=trainers, y=labels)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [158]:
trainers.shape[1], predictors.shape[1]

(265, 264)

In [132]:
predicts = model3.predict(X=predictors)

ValueError: shapes (26860,270) and (271,100) not aligned: 270 (dim 1) != 271 (dim 0)

In [112]:
soloq_df['position'] = predicts

In [113]:
results = {1: {'predicts': predicts, 'counts': soloq_df.position.value_counts(), 'model': 'MLPClassifier', }}

In [117]:
results

{'3': {'counts': MID     6192
  ADC     5880
  TOP     5558
  JUNG    4896
  SUPP    4334
  Name: position, dtype: int64,
  'model': 'MLPClassifier',
  'predicts': array(['MID', 'MID', 'JUNG', ..., 'ADC', 'JUNG', 'TOP'], 
        dtype='<U4')}}

In [116]:
soloq_df[['gameId', 'summonerName', 'champ_name', 'side', 'lane', 'role', 'position']]

,gameId,summonerName,champ_name,side,lane,role,position
0,3770407781,D1 WARRIOR,Kog'Maw,Blue,MIDDLE,DUO_SUPPORT,MID
1,3770407781,Kyrîel,Braum,Blue,MIDDLE,DUO_SUPPORT,MID
2,3770407781,ChosenØne,Graves,Blue,JUNGLE,NONE,JUNG
3,3770407781,Decak,Cho'Gath,Blue,MIDDLE,DUO_SUPPORT,TOP
4,3770407781,MidLurk3r,Cassiopeia,Blue,MIDDLE,DUO,TOP
5,3770407781,ordno,Galio,Red,MIDDLE,SOLO,TOP
6,3770407781,Archie2b,Gragas,Red,JUNGLE,NONE,JUNG
7,3770407781,ºˆºˆºˆºˆºˆºˆºˆº,Alistar,Red,BOTTOM,DUO_SUPPORT,SUPP
8,3770407781,Nawada,Kai'Sa,Red,BOTTOM,DUO_CARRY,TOP
9,3770407781,DXZOREN,Illaoi,Red,TOP,SOLO,TOP


# Predictions with only champion, role and lane info 